In [3]:
import numpy as np 
import sciann as sn
import matplotlib.pyplot as plt 

tunits = 3
x = sn.functionals.RNNVariable(tunits, 'x', dtype='float64')
y = sn.functionals.RNNVariable(tunits, 'y', dtype='float64')
t = sn.functionals.RNNVariable(tunits, 't', dtype='float64')

P = sn.functionals.RNNFunctional("P", [x, y, t], 8*[20], 'tanh')
Psi = sn.functionals.RNNFunctional("Psi", [x, y, t], 8*[20], 'tanh')

lambda1 = sn.Parameter(np.random.rand(), inputs=[x,y,t], name="lambda1")
lambda2 = sn.Parameter(np.random.rand(), inputs=[x,y,t], name="lambda2")

u = sn.diff(Psi, y)
v = -sn.diff(Psi, x)

u_t = sn.diff(u, t)
u_x = sn.diff(u, x)
u_y = sn.diff(u, y)
u_xx = sn.diff(u, x, order=2)
u_yy = sn.diff(u, y, order=2)

v_t = sn.diff(v, t)
v_x = sn.diff(v, x)
v_y = sn.diff(v, y)
v_xx = sn.diff(v, x, order=2)
v_yy = sn.diff(v, y, order=2)

p_x = sn.diff(P, x)
p_y = sn.diff(P, y)

d1 = sn.Data(u)
d2 = sn.Data(v)
d3 = sn.Data(P)

c1 = sn.Tie(-p_x, u_t+lambda1*(u*u_x+v*u_y)-lambda2*(u_xx+u_yy))
c2 = sn.Tie(-p_y, v_t+lambda1*(u*v_x+v*v_y)-lambda2*(v_xx+v_yy))
c3 = sn.Data(u_x + v_y)
c4 = Psi*0.0

model = sn.SciModel(
    inputs=[x, y, t],
    targets=[d1, d2, d3, c1, c2, c3, c4],
    loss_func="mse",
)

from tqdm import tqdm

import os
import numpy as np

import scaling
import data

USE_2D = False
INPUTS = ['X', 'Y']
OUTPUTS = ['Vu', 'Vv', 'P']
PREDICT_AHEAD=1
NEIGHBOR_SIZE=1
LOOKBACK = 1
BATCH_SIZE = 5024

SCALER_PATH = os.path.join('output', f'scaler_{INPUTS}_2D={USE_2D}.pkl')
SCALER_CREATION_DIRS = ['/home/jperez/data/sled250', '/home/jperez/data/sled255']

sc = scaling.load_or_create(SCALER_PATH, SCALER_CREATION_DIRS, INPUTS, OUTPUTS, USE_2D)

train_generator = data.SledDataGenerator('/home/jperez/data/sled250', batch_size=BATCH_SIZE, lookback=LOOKBACK, predict_ahead=PREDICT_AHEAD, neighbor_size=NEIGHBOR_SIZE, shuffle=True, use_2D=USE_2D, inputs=INPUTS, outputs=OUTPUTS, scaler=sc, 
                                        start=1, end=510+1)
train_generator.sciann = True

history = model.train(
    x_true=train_generator,
    epochs=10,
    shuffle=True,
    learning_rate=0.001,
    reduce_lr_after=100,
    stop_loss_value=1e-8,
)

TypeError: 

In [ ]:
def PrepareData(num_data=5000, random=True):
    import scipy.io
    # Get data file from: 
    #         https://github.com/maziarraissi/PINNs/tree/master/main/Data/cylinder_nektar_wake.mat
    data = scipy.io.loadmat('cylinder_nektar_wake.mat')
    
    U_star = data['U_star'] # N x 2 x T
    P_star = data['p_star'] # N x T
    t_star = data['t'] # T x 1
    X_star = data['X_star'] # N x 2
    
    N = X_star.shape[0]
    T = t_star.shape[0]
    
    # Rearrange Data 
    XX = np.tile(X_star[:,0:1], (1,T)) # N x T
    YY = np.tile(X_star[:,1:2], (1,T)) # N x T
    TT = np.tile(t_star, (1,N)).T # N x T
    
    UU = U_star[:,0,:] # N x T
    VV = U_star[:,1,:] # N x T
    PP = P_star # N x T
    
    # Pick random data.
    if random:
        idx = np.random.choice(N*T, num_data, replace=False)
    else:
        idx = np.arange(0, N*T)
    
    x = XX.flatten()[idx,None] # NT x 1
    y = YY.flatten()[idx,None] # NT x 1
    t = TT.flatten()[idx,None] # NT x 1
    
    u = UU.flatten()[idx,None] # NT x 1
    v = VV.flatten()[idx,None] # NT x 1
    p = PP.flatten()[idx,None] # NT x 1
 
    return (x,y,t,u,v,p)

In [ ]:
x_train, y_train, t_train, u_train, v_train, p_train = PrepareData(5000, random=True)
print(x_train.shape, t_train.shape, p_train.shape)

(5000, 1) (5000, 1) (5000, 1)


In [5]:


# ALL_X = []
# ALL_Y = []
# t_train = []

# for timestep in tqdm(range(1, 638+1)):
#     X, Y = data.read_np(os.path.join('/home/jperez/data/sled250', f'{timestep}.npy'), INPUTS, OUTPUTS, use_2D=False, scaler=sc)
    
#     ALL_X.append(X)
#     ALL_Y.append(Y)

#     tarr = np.zeros((14184, 1))
#     tarr[:] = timestep
#     t_train.append(tarr)

# print(len(ALL_X[0]))
# ALL_X = np.array(ALL_X).reshape(-1, len(INPUTS))
# ALL_Y = np.array(ALL_Y).reshape(-1, len(OUTPUTS))
# t_train = np.array(t_train).reshape(-1, 1)

# x_train = ALL_X[:, 0].reshape(-1, 1)
# y_train = ALL_X[:, 1].reshape(-1, 1)
# u_train = ALL_Y[:, 0].reshape(-1, 1)
# v_train = ALL_Y[:, 1].reshape(-1, 1)
# p_train = ALL_Y[:, 2].reshape(-1, 1)

Loading previous scaler


100%|██████████| 638/638 [00:00<00:00, 2056.19it/s]


14184


In [6]:
print(f'{(ALL_X.nbytes+ALL_Y.nbytes)/1e9:.3f}GB of RAM used')
print(ALL_X.shape, ALL_Y.shape)
print(x_train.shape, y_train.shape, t_train.shape, u_train.shape, v_train.shape, p_train.shape)

0.362GB of RAM used
(9049392, 2) (9049392, 3)
(9049392, 1) (9049392, 1) (9049392, 1) (9049392, 1) (9049392, 1) (9049392, 1)


In [7]:
input_data = [x_train, y_train, t_train]

data_d1 = u_train
data_d2 = v_train
data_d3 = p_train
data_c1 = 'zeros'
data_c2 = 'zeros'
data_c3 = 'zeros'
data_c4 = 'zeros'
target_data = [data_d1, data_d2, data_d3, data_c1, data_c2, data_c3, data_c4]

In [8]:
history = model.train(
    x_true=input_data,
    y_true=target_data,
    epochs=10,
    batch_size=50000,
    shuffle=True,
    learning_rate=0.001,
    reduce_lr_after=100,
    stop_loss_value=1e-8,
)


Total samples: 9049392 
Batch size: 50000 
Total batches: 181 

Epoch 1/10
181/181 [==============================] - 99s 452ms/step - batch: 90.0000 - size: 49996.6409 - loss: 1626258388685.7427 - Grad__loss: 25165.5630 - mul_loss: 17243.8531 - P_loss: 1626258346264.6731 - sub_2_loss: 8.4958 - sub_4_loss: 3.1578 - add_7_loss: 6.2228e-29 - mul_12_loss: 0.0000e+00
Epoch 2/10
181/181 [==============================] - 82s 455ms/step - batch: 90.0000 - size: 49996.6409 - loss: 1626282778084.9138 - Grad__loss: 22866.7400 - mul_loss: 16172.1966 - P_loss: 1626282739038.6201 - sub_2_loss: 4.3754 - sub_4_loss: 2.9824 - add_7_loss: 3.7723e-28 - mul_12_loss: 0.0000e+00
Epoch 3/10
181/181 [==============================] - 83s 457ms/step - batch: 90.0000 - size: 49996.6409 - loss: 1626262395324.4712 - Grad__loss: 22238.2944 - mul_loss: 15594.1816 - P_loss: 1626262357484.0249 - sub_2_loss: 3.4817 - sub_4_loss: 4.4890 - add_7_loss: 5.9226e-28 - mul_12_loss: 0.0000e+00
Epoch 4/10
181/181 [=========

KeyboardInterrupt: 

In [ ]:
print("lambda1: {},  lambda2: {}".format(lambda1.value, lambda2.value))

In [ ]:
plt.semilogy(history.history['loss'])
plt.xlabel('epochs')
plt.ylabel('loss')